<a href="https://colab.research.google.com/github/War-rack/PillBot/blob/main/Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio opencv-python numpy torch torchvision transformers paddleocr nltk symspellpy matplotlib albumentations paddlepaddle pillow

import gradio as gr
import cv2
import numpy as np
import torch
import re
import pkg_resources
import nltk
from nltk.corpus import words
from symspellpy import SymSpell
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from paddleocr import PaddleOCR

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize components
nltk.download("words")
english_words = set(words.words())

# Custom medical dictionary and spell checker setup
sym_spell = SymSpell(max_dictionary_edit_distance=2)
sym_spell.load_dictionary(pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt"),
                         term_index=0, count_index=1)

MEDICAL_TERMS = {
    'moxigram', 'trehalube', 'fluocinolone', 'acetonide', 'neomycin',
    'beclomethasone', 'clotrimazole', 'miconazole', 'clobetasol', 'propionate',
    'cetirizine', 'tobetsol', 'lx', 'eye', 'drop', 'drops', 'ointment', 'cream'
}

# Add medical terms with high priority
for term in MEDICAL_TERMS:
    sym_spell.create_dictionary_entry(term, 1000000)

# Load OCR models
trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten").to(device)
paddle_ocr = PaddleOCR(use_angle_cls=True, lang="en")

# OCR processing functions
def extract_text_vit(image):
    pixel_values = trocr_processor(image, return_tensors="pt").pixel_values.to(device)
    with torch.no_grad():
        generated_ids = trocr_model.generate(pixel_values)
    return trocr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

def extract_text_paddleocr(image):
    results = paddle_ocr.ocr(np.array(image), cls=True)
    if not results or not results[0]:
        return ""
    return " ".join([res[1][0] for res in results[0] if res[1][1] > 0.5])

def extract_medicines(text):
    medicine_patterns = [
        r'\b(?:[a-z]+\s){1,3}(?:eye\s?drops?|ointment|cream)\b',
        r'\b[a-z]+\s(?:[a-z]+\s)?[a-z]+\b',
        r'\b[a-z]+\b(?=\s*\d+%?)'
    ]
    found_meds = set()
    for pattern in medicine_patterns:
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        for match in matches:
            cleaned = re.sub(r'\s*(?:eye\s?drops?|drops|ointment|cream)\b', '', match, flags=re.I)
            cleaned = re.sub(r'\s+', ' ', cleaned).strip().title()
            if len(cleaned) > 3:
                found_meds.add(cleaned)
    return sorted(found_meds)

def identify_medicines(image):
    vit_text = extract_text_vit(image)
    paddle_text = extract_text_paddleocr(image)
    combined_text = f"{vit_text} {paddle_text}"
    medicines = extract_medicines(combined_text)
    return "\n".join(medicines) if medicines else "No medicines identified."

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Medicine Identification using OCR")

    with gr.Tab("Upload Image"):
        image_input = gr.Image(type="pil")
        output_text = gr.Textbox(label="Identified Medicines")
        submit_btn = gr.Button("Process Image")
        submit_btn.click(identify_medicines, inputs=image_input, outputs=output_text)

    with gr.Tab("Medicine Reminder Scheduler"):
        gr.Markdown("[Go to Medicine Reminder Scheduler](http://localhost:3001)")

    with gr.Tab("Advanced OCR Processing"):
        advanced_image_input = gr.Image(type="filepath")
        advanced_output_text = gr.Textbox(label="Extracted Medicines")
        process_btn = gr.Button("Run Advanced OCR")
        process_btn.click(identify_medicines, inputs=advanced_image_input, outputs=advanced_output_text)

demo.launch(share=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
  

<ipython-input-1-4ceb2d58cfd8>:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

...

100%|██████████| 10000/10000 [00:19<00:00, 518.22it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:15<00:00, 134.23it/s]

[2025/03/07 06:54:20] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74082de09bb7d44b86.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
